In [1]:
# 1.csv Jan 1, 2013- April 5, 2022; 
# 2.csv Jan 1, 2003 – December 31, 2012 
# 3.csv Jan 1, 1993 - December 31, 2002 

In [55]:
import pandas as pd
import numpy as np
import math
from datetime import datetime

In [56]:
import sqlite3
conn = sqlite3.connect('database.db')
cur = conn.cursor()

In [57]:
shortened_date = lambda d : datetime.strptime(d[:10], '%Y-%m-%d')
set_month_year = lambda d : str(d.year)+str(d.month)
set_year = lambda d : d.year
set_month = lambda d : d.month

In [58]:
ncd_cols = ['DATE', 'REPORT_TYPE', 
            'DailyAverageDryBulbTemperature', 
            'DailyAverageStationPressure',
            'DailyAverageWindSpeed', 
            'DailyCoolingDegreeDays', 'DailyDepartureFromNormalAverageTemperature', 
            'DailyHeatingDegreeDays',
            'DailyMaximumDryBulbTemperature', 'DailyMinimumDryBulbTemperature', 
            'DailyPeakWindDirection', 'DailyPeakWindSpeed', 
            'DailyPrecipitation', 
            'DailySustainedWindDirection', 'DailySustainedWindSpeed',
            'DailyWeather']

In [59]:
df1 = pd.read_csv('datasets/nenana_climate/1.csv', usecols = ncd_cols) 
df2 = pd.read_csv('datasets/nenana_climate/2.csv', usecols = ncd_cols) 
df3 = pd.read_csv('datasets/nenana_climate/3.csv', usecols = ncd_cols) 


/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (21,27,28,29,30,31,32,33,37,38) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (21,27,28,29,30,31,33,34,38,39) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [60]:
df1['DATE'] = df1['DATE'].apply(shortened_date)
df2['DATE'] = df2['DATE'].apply(shortened_date)
df3['DATE'] = df3['DATE'].apply(shortened_date)


In [61]:
# SOD sum of day, delete hourly records 
df1 = df1[df1['REPORT_TYPE']=='SOD  '].set_index('DATE').drop('REPORT_TYPE',1) 
df2 = df2[df2['REPORT_TYPE']=='SOD  '].set_index('DATE').drop('REPORT_TYPE',1) 
df3 = df3[df3['REPORT_TYPE']=='SOD  '].set_index('DATE').drop('REPORT_TYPE',1) 


In [62]:
nenana_climate = pd.concat([df1,df2,df3]).reset_index()
nenana_climate['year'] = nenana_climate['DATE'].apply(set_year)
nenana_climate['month'] = nenana_climate['DATE'].apply(set_month)


In [63]:
years_index = sorted([y for y in nenana_climate['year'].unique()])
months_index = sorted([m for m in nenana_climate['month'].unique()])

In [64]:
nenana_climate['DATE'].max()

Timestamp('2022-01-01 00:00:00')

In [101]:
t= dict()
for y in years_index:
    t[y] = {}
    for m in months_index:
        t[y][m] =[]
        for v in nenana_climate[ (nenana_climate['year']==y) & (nenana_climate['month']==m) ].DailyWeather.values:
            if type(v)== str: 
                t[y][m]+= [y for y in v.split() ]

In [154]:
weather_types = pd.DataFrame.from_dict(t)
weather_types

,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
1,"[SN, SN, FG, FG, FG, FG, FG, FG, SN, SN, SN]","[BR, BR, SN, SN, BR]","[SN, SN, SN, BR, FG, BR, HZ, SN, BR, HZ, HZ, S...","[SN, SN, BR, HZ, BR, SN, BR, SN, BR, SN, BR, S...",[],[],[],[],"[HZ, SN, BR, SN, BR, SN, BR, HZ, BR, HZ, BR, H...","[SN, UP, SN, BR, UP, SN, BR, UP, SN, FZFG, BR,...",...,"[RA, BR, RA, RA, RA, RA, RA, SN, BR, RA, SN, H...","[RA, SN, RA, SN, SN, RA, RA, RA, SN, HZ, FZFG,...","[SN, BR, SN, SN, RA, RA, BR, RA, UP, RA, RA, S...","[BR, HZ, RA, RA, RA, SN, FZRA, SN, BR, HZ, RA,...","[SN, BR, SN, BR, SN, BR, UP, BR, SN, SN, BR, S...","[BR, BR, UP, FZRA, SN, BR, RA, FZRA, SN, FZFG,...","[RA, SN, FZFG, BR, UP, FG, BR, UP, HZ, HZ, BR,...","[SN, BR, UP, HZ, SN, HZ, HZ, SN, HZ, BR, HZ, B...","[HZ, SN, BR, UP, UP, BR, HZ, HZ, HZ, SN, SN, HZ]","[SN, UP, HZ]"
2,"[FG, FG, FG, FG, SN, FG, SN, SN, SN, SN, SN, SN]","[BR, SN, BR, BR, SN]","[SN, BR, HZ, RA, SN, BR, SN, BR, SN]","[SN, SN, HZ, SN, BR, HZ, SN, BR, HZ, SN, BR, H...",[],[],[],[],"[SN, BR, SN, BR, HZ, HZ, SN, BR, SN, SN, HZ, S...","[BR, HZ, BR, HZ, BR, UP, HZ, BR, HZ, BR, HZ, U...",...,"[RA, RA, RA, SN, BR, RA, BR, RA, SN, HZ, RA, S...","[UP, SN, BR, SN, BR, HZ, FZRA, SN, FZFG, BR, U...","[SN, BR, RA, HZ, RA, SN, RA, RA, RA, RA, HZ, R...","[RA, UP, RA, UP, SN, HZ, FZFG, BR, HZ, RA, FG,...","[FZFG, BR, UP, HZ, FG, FZFG, BR, HZ, FG, HZ, U...","[RA, RA, SN, BR, RA, BR, RA, SN, BR, UP, HZ, S...","[SN, BR, SN, BR, SN, BR, UP, RA, SN, HZ, BR, S...","[HZ, SN, HZ, SN, SN, FZFG, BR, HZ, FG, SN, BR,...","[SN, SN, HZ, SN, SN, SN, FG, BR, FZFG, BR, HZ,...",[]
3,"[SN, SN, SN, SN, SN, RA]","[SN, SN, SN]","[SN, SN, BR, SN, BR, SN, BR, SN, SN, SN]","[SN, BR, SN, BR, SN, BR, HZ, HZ, SN, SN, SN, H...",[],[],[],[],"[SN, BR, HZ, SN, BR, SN, FG, BR, HZ, SN, FG, B...","[SN, BR, UP, SN, UP, SN, UP, UP, UP, RA, UP, U...",...,"[SN, BR, RA, HZ, RA, BR, RA, SN, FZFG, BR, FG,...","[UP, SN, BR, HZ, SN, BR, HZ, SN, UP, UP, RA, U...","[SN, BR, RA, SN, BR, SN, BR, UP, SN, BR, UP, H...","[RA, SN, SN, SN, BR, HZ, HZ, UP, SN, HZ, RA, R...","[RA, RA, SN, SN, BR, UP, HZ, SN, BR, HZ, SN, B...","[HZ, SN, BR, UP, SN, FZFG, BR, FG, SN, BR, UP,...","[SN, BR, SN, BR, SN, BR, HZ, SN, BR, RA, SN, B...","[SN, SN, HZ, SN, HZ, HZ, HZ, SN, HZ, SN, HZ, S...","[SN, SN, SN, SN, HZ, SN, BR, HZ, SN, BR, SN, H...",[]
4,"[SN, SN, RA]","[SN, SN, SN, RA]","[SN, BR, RA, RA, SN, BR, SN, BR, SN]","[SN, BR, SN, BR, SN, FG, SN, RA, SN, BR]",[],[],[],[],"[SN, SN, SN, BR, RA, RA, RA]","[SN, UP, SN, BR, UP, SN, BR, UP, SN, BR, UP, U...",...,"[SN, BR, RA, SN, SN, BR, HZ, SN, BR, HZ, RA, S...","[RA, RA, RA, RA, SN, BR, HZ, RA, SN, BR, RA, R...","[SN, HZ, RA, SN, RA, SN, BR, HZ, UP, SN, SN, B...","[SN, BR, SN, BR, UP, RA, RA, RA, RA, SN, BR, H...","[RA, RA, HZ, RA, RA, SN, BR, RA, SN, BR, UP, H...","[SN, BR, SN, RA, RA, RA, RA, RA, RA, RA, RA, R...","[RA, RA, RA, RA, RA, RA, BR, HZ, RA, RA, SN, B...","[SN, BR, HZ, RA, SN, FG, BR, UP, RA, SN, UP, H...","[SN, UP, HZ, SN, BR, SN, FZFG, BR, FG, RA, SN,...",[]
5,"[RA, FG, RA, RA]","[RA, RA, RA, RA, RA, RA]","[RA, SN, BR, SN, BR, RA, RA, RA, RA, BR, RA, R...","[SN, BR, RA, RA, RA, FG, BR, RA, RA, RA, FG, B...",[],[],[],[],"[RA, BR, RA, BR, RA, SN, BR, RA, BR, RA, BR, R...","[RA, HZ, RA, BR, HZ, BR, BR]",...,"[SN, BR, HZ, RA, SN, RA, BR, SN, FZFG, BR, UP,...","[RA, RA, RA, RA, HZ, BR, RA, RA, RA, RA, RA, R...","[RA, BR, BR, RA, RA, RA]","[RA, RA, BR, RA, RA, RA, RA, RA, BR, RA, RA, R...","[RA, RA, RA, RA, RA, RA, RA, RA, RA, BR, RA, R...","[RA, SN, BR, UP, SN, BR, RA, RA, RA, BR, RA, S...","[RA, SN, BR, RA, RA, RA, RA, RA, RA, RA, RA, F...","[RA, RA, RA, RA]","[RA, RA, RA, RA, RA, RA, RA, RA, RA]",[]
6,"[RA, SN]",[RA],"[RA, RA, RA, RA, RA, RA, RA, FG, BR, RA, FG, B...","[RA, BR, RA, RA, RA, RA, RA, BR, RA, BR, RA, F...",[],[],[],[],"[RA, RA, BR, FG, BR, RA, BR, RA, RA, RA, FG, B...","[HZ, BR, FG, BR, HZ, BR, HZ, BR, BR, BR, HZ, H...",...,"[RA, RA, RA, RA, RA, RA, RA, RA, RA, RA, HZ, R.

In [130]:
ind, col, val = [], [], []
for k,v in weather_types.items():
    ind.append(k)
    for k2, v2 in v.items():
        col.append(k2)
        val.append(v2)

In [137]:
col

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,

In [153]:
weather_types = pd.DataFrame.from_dict(weather_types);
weather_types

""
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005


In [155]:
weather_types.to_sql('weathertypes', conn, if_exists='replace')

InterfaceError: Error binding parameter 1 - probably unsupported type.